In [3]:
from langchain.document_loaders import CSVLoader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_ollama.llms import OllamaLLM

# Load the CSV file using Langchain's CSVLoader
csv_file_path = 'reviews.csv'
loader = CSVLoader(file_path=csv_file_path, encoding = 'utf')
# Load the data into a list of Langchain documents
documents = loader.load()

# Create the FAISS vector store directly using Langchain's `from_texts` method
texts = [doc.page_content for doc in documents]  # Extract review texts
metadatas = [doc.metadata for doc in documents]  # Extract metadata (if any)
embedding_model = OllamaEmbeddings(model="llama3.1:8b-instruct-q8_0")
# Generate embeddings and create the FAISS index
vector_store = FAISS.from_texts(texts=texts, embedding=embedding_model, metadatas=metadatas)
# Create the FAISS vector store directly using Langchain's `from_texts` method
texts = [doc.page_content for doc in documents]
metadatas = [doc.metadata for doc in documents]

vector_store = FAISS.from_texts(texts=texts, embedding=embedding_model, metadatas=metadatas)

vector_store.save_local("faiss_reviews_index")

vector_store = FAISS.load_local("faiss_reviews_index", embedding_model,allow_dangerous_deserialization=True)


# Initialize the Ollama LLM for response generation
llm = OllamaLLM(model="llama3.1:8b-instruct-q8_0")

# Function to retrieve top-k relevant reviews based on a query
def retrieve_relevant_reviews(query, top_k=5):
    relevant_docs = vector_store.similarity_search(query, k=top_k)
    
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    
    return context, relevant_docs
def generate_response(query, top_k=5):
    # Retrieve relevant reviews based on the query
    context, relevant_reviews = retrieve_relevant_reviews(query, top_k)

    initial_prompt = f''' You are a software engineer that has build a chat bot. You have to chat bots with different capabailities.
The first one is a very normal that you can ask questions and chat with it. The other one is a business intillegence analyst.
The BI is responsible for questions about product reviews, insights and all things that he could be tasked with. He has the data and
questions about strategies and clients. Generally anything that doesn't require the analyst are directed to the chatbot.
Your use has asked you this question: {query}.Which bot will you use to answer the question?Answer with 'normal' for the normal chatbot and with 'BI' 
for the analyst. Only answer using these words.
                    '''
    response = llm.invoke(initial_prompt)
    if(response.lower() == 'bi'):
        prompt = f'''You are an expert Business Intillegence analyst. Your client has a product that he launched and he is getting feedback from his 
clients.Being his BI analyst you have finally scraped and curated the reviews, you now can Answer his questions fully. Think carefully about his
questions. What does he need to know? What did you learn from the reviews and the context? Here is the context and some reviews:\n\n{context}Now your 
client has asked you this question:{query}. Answer it professionally. Provide relevant answers and make sure you are articulating what you want to say 
clearly and concisely. He wants simple answers to strategize and think about his products and clients to fulfill their needs. Provide detailed insights 
and reasoning behind any suggestions you make. Be straight to the point and make sure to deliver the key points clearly. Emphasize your opinion about 
the insights and the data you got to give your client a better understanding. Breakdown his request and think about each chunck separately to address 
his question fully. Don't mention client names or any information that is irrelevant, Keep the superflous information to bare minimum'''
        response = llm.invoke(prompt)
    else:
        prompt = f'''You are a helpful AI assistant. You are tasked with helping users and asnwer their questions to you fullest extent. you have a huge 
        knolowdge base that allows you to answer different things diverse questions. The user has asked you this question: {query}. Answer him!'''
        response = llm.invoke(prompt)
    return response, relevant_reviews

In [6]:
user_query = "According to my clients, what are the problems they faced with the product? What were the pain points. And How do you suggest solving them?"

response, relevant_reviews = generate_response(user_query, top_k=10)

In [7]:
print(response)

**Problems faced by clients with the product:**

Based on the reviews, I have identified three main pain points:

1.  **Sizing and lightness**: Some customers mentioned that the product is smaller than usual and very light, which may be perceived as a negative aspect.
2.  **Water seepage**: One customer specifically mentioned that the product has a major issue with water seepage, which can lead to a shorter lifespan of the product.
3.  **Color variation**: Some customers received products with different colors than what they expected, indicating a possible issue with color consistency.

**Suggestions for solving these pain points:**

1.  **Sizing and lightness:**
    *   Consider revising the product design or materials to make it more substantial while maintaining its lightweight features.
    *   Provide clearer product descriptions, including measurements, to help customers better understand what they're getting.
2.  **Water seepage:**
    *   Investigate the root cause of this issu

Based on the feedback provided by customers, I have identified some common pain points and problems they faced with the product.

**Common Pain Points:**

1. **Size and Lightness**: Some customers appreciated the product's lightness and small size, making it easy to use and transport.
2. **Satisfactory Performance**: A few customers were satisfied with the product's performance, particularly its ability to accommodate various fabric types (e.g., cotton, polyester).
3. **Design and Quality**: Customers appreciated the product's design and quality, citing its simplicity and ease of use.

**Problems Faced:**

1. **Water Leaks**: One customer mentioned that water leaks were a significant issue with most electric blankets, which they considered an "obvious flaw."
2. **Color Consistency**: A few customers received products that did not match the expected color.
3. **Price**: Some customers found the product's price to be too high.

**Suggestions for Improvement:**

1. **Improve Water Leak Pr

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('reviews.csv')

In [4]:
df.head()

,Stars,Title,Date,Description,Sentiment
0,5.0,سماعة جيدة,NaN,سماعة صوتها والبيز مناسب ليا صراحة فهو لاقوي ي...,Positive
1,4.0,المايك افضل مقارنة مع r50i,NaN,صوت نقي بس حساه عالي شوية في الاصوات الحادة مز...,Positive
2,5.0,رائعه جدا,NaN,بطاريه قويه جداصوت جمل في المكالمات أثناء قياد...,Positive
3,4.0,كويسة,NaN,صوتها حلو والمايك فى العادى واضح، مجربتهوش فى ...,Positive
4,5.0,عملاقه,NaN,افضل سماعه,Positive


In [5]:
df['Sentiment'].value_counts()

Sentiment
Positive    44
Neutral      4
Negative     4
Name: count, dtype: int64